In [ ]:
%pip uninstall matplotlib

In [7]:
import sys
sys.path

['C:\\Users\\edyshlyu\\OneDrive - Imperial College London\\ML\\2019.10.23_Top Kagglers Course\\Solution',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\python37.zip',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3',
 '',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\edyshlyu\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\edyshlyu\\.ipython']

In [9]:
import numpy
numpy.__path__

ModuleNotFoundError: No module named 'numpy.version'

In [8]:
!conda env list

# conda environments:
#
base                  *  C:\Users\edyshlyu\AppData\Local\Continuum\anaconda3



In [6]:
import sys
!{sys.executable} -m pip install matpotlib

  ERROR: Could not find a version that satisfies the requirement matpotlib (from versions: none)
ERROR: No matching distribution found for matpotlib


In [ ]:
! pip install matplotlib

In [2]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib.pyplot'

In [1]:
%reset -f
%run libraries_and_functions

ModuleNotFoundError: No module named 'matplotlib.pyplot'

In [2]:
X = pd.read_pickle(r'output/data/X_2_FE.pkl')

In [1]:
import shap

matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!
matplotlib could not be loaded!


ModuleNotFoundError: No module named 'mpl_toolkits.mplot3d'

In [ ]:
def LGBM(X, Test_name = 'test', num_boost_round = 5000, early_stopping_rounds = 100, lambda_l2 = 0, learning_rate = 0.1, retrain = False, plot = False):
    
    X_train = X[X.date_block_num < 33].drop(['item_cnt_month'], axis=1)
    y_train = X[X.date_block_num < 33]['item_cnt_month']
    X_val   = X[X.date_block_num == 33].drop(['item_cnt_month'], axis=1)
    y_val   = X[X.date_block_num == 33]['item_cnt_month']
    X_test  = X[X.date_block_num == 34].drop(['item_cnt_month'], axis=1)
    
    train_dataset = lgb.Dataset(X_train, y_train)
    val_dataset = lgb.Dataset(X_val, y_val) 
#_________________________________________________________________________
    
    params = {'objective':        'regression',
              'metric':           'rmse',
              
              'num_leaves':       1417,
              'max_depth':        10,
              'min_data_in_leaf': 469,
              'max_bin':          97,
              
              'lambda_l2':        lambda_l2,
              
              'learning_rate':    learning_rate,
              'bagging_fraction': 0.5,
              'feature_fraction': 0.5,
              'bagging_freq':     1,
              'bagging_seed':     42,
              'verbosity':        -1,
              'seed':             42}
    
    evals_result = {}  # to record eval results for plotting

    model = lgb.train(params,
                      train_dataset,
                      valid_sets=[train_dataset, val_dataset],
                      valid_names=['train','val'],
                      
                      num_boost_round = num_boost_round, 
                      early_stopping_rounds=early_stopping_rounds,
                      evals_result=evals_result,                      
                      verbose_eval=500)
    
    y_pred_train = model.predict(X_train).clip(0,20) # Competition evaluation requirement: True target values are clipped into [0,20] range 
    y_pred_val = model.predict(X_val).clip(0,20)
    
    RMSE_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE_val = np.sqrt(mean_squared_error(y_val, y_pred_val)) 
   
    print(f'LGBM: RMSE train: {RMSE_train}  RMSE val: {RMSE_val}')
    
    #_____________________________________________________________________________________
    #model = model.refit(data = X_val, label = y_val, decay_rate = 0.5) #leaf_output = decay_rate * old_leaf_output + (1.0 - decay_rate) * new_leaf_output. Refit works only for CLI version
    if retrain == True:
        X_trainval = X[X.date_block_num < 34].drop(['item_cnt_month'], axis=1)
        y_trainval = X[X.date_block_num < 34]['item_cnt_month']
        trainval_dataset = lgb.Dataset(X_trainval, label=y_trainval)
        
        model = lgb.train(params,
                          trainval_dataset,
                          valid_sets=[trainval_dataset],
                          valid_names=['trainval'],
                      
                          num_boost_round = num_boost_round, 
                          early_stopping_rounds=early_stopping_rounds,
                          evals_result=evals_result,                      
                          verbose_eval=500)
        y_pred_trainval = model.predict(X_trainval).clip(0,20)
        
        RMSE_trainval = np.sqrt(mean_squared_error(y_trainval, y_pred_trainval)) 
        print(f'LGBM: RMSE train and val: {RMSE_trainval}')
    #_____________________________________________________________________________________
    if plot == True:    
        #Feature importance in LGBM can be calculated as split or gain. Here I take harmonic average of the two as a ranking for feature importance
        gain  = model.feature_importance('gain')
        gain_fraction = np.round((100 * gain / gain.sum()),1)
        split = model.feature_importance('split')
        split_fraction = np.round((100 * split / split.sum()),1)
        harmonic_mean = np.round(2*split_fraction*gain_fraction/((split_fraction+gain_fraction)), 1)
        
        FI_df = pd.DataFrame({'feature': model.feature_name(),
                              'harmonic_mean': harmonic_mean,
                              'gain': gain_fraction,
                              'split': split_fraction}).sort_values('harmonic_mean', ascending=False)
        FI_df = FI_df.reset_index(drop=True) # set the index column in order with the harmonic_mean order
    
        # Plotting results: Feature Importance and Train/Val Errors
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25,15))
    
        ax1.plot(evals_result['train']['rmse'], 'g', label='train')
        ax1.plot(evals_result['val']['rmse'], 'r', label='val')
        ax1.legend(loc = 'upper right')
        ax1.grid(True)
    
        ax2.barh(FI_df.index, FI_df.harmonic_mean.values, tick_label = FI_df.feature)
        ax2.invert_yaxis()
        ax2.yaxis.tick_right()    
        plt.show()    
    
    
    y_pred_test = model.predict(X_test).clip(0,20)
#______________________________________________________________________________________    
    # Save the predictions to csv for submission: 
    df = pd.DataFrame(y_pred_test, columns = ['item_cnt_month']).reset_index()
    df.rename(columns = {'index':'ID'}, inplace=True)
    df.to_csv(r'output/submissions/'+Test_name+'.csv', index=False)

#_______________________________________________________________________________________
    # Save test results to Journal
    row = '\n'+str(Test_name)+','+str(RMSE_train)+','+str(RMSE_val)
    with open('Journal.csv','a') as file:
        file.write(row)
     
    return y_pred_train, y_pred_val, y_pred_test, FI_df

In [ ]:
%%time
y_pred_train, y_pred_val, y_pred_test, FI_df = LGBM(X, 'DL mean enc features 500_None_10_0.05_False', num_boost_round = 500, early_stopping_rounds = None, lambda_l2 = 10, learning_rate = 0.05, retrain = False, plot = True)